In [1]:
import pyodbc
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import logging

from texttransformation import StringTransform, RowTextTransform, TransformDataset
from datasetbuilder import DatasetBuilder
from metrics import MetricsCalculator
from helpers import SaveModel, RestoreModel, Normalize, RemoveOutliers
from predictor import Predictor

Configure parameters 

In [2]:
DATASOURCE_COLUMNS = ['FindCode', 'Name', 'Phone', 'Fax', 'OtherPhone', \
                                  'Email', 'WebsiteURL', 'MailingAddressFreeform', \
                                  'MailingAddressCity', 'MailingAddressPostalCode', 'MailingAddressState']
DATASOURCE_INDEX = 'FindCode'
ONE_HOT_ENCONDING_COLUMNS = {'MailingAddressState': ['nunavut ',
                                                        'saskatchewan',
                                                        'ontario  ',
                                                        'alberta',
                                                        'british columbia ',
                                                        'prince edward island ',
                                                        'yukon territory',
                                                        'newfoundland',
                                                        'northwest territories',
                                                        'new brunswick',
                                                        'manitoba',
                                                        'nova scotia ',
                                                        'quebec']}
TEXT_METRICS = ['ratio','partial_ratio','token_sort_ratio','token_set_ratio','distance',
                'l_ratio','jaro','jaro_winkler','setratio','seqratio','longestnumericseq']
PASS_THROUGH_COMULNS = ['FindCode','MailingAddressState']
HIGH_IMPORTANCE_COLUMNS = ['Name']

alteration_rules = [
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['alpha', 2, 'replace'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [2],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['any', 2, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [4],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['digits', 1, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [3],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    }
]

COLUMN_ALTERATIION_RULES = []
for i in range(0, len(alteration_rules)):
    COLUMN_ALTERATIION_RULES.append(
        {
            1: alteration_rules[i],
            2: alteration_rules[i],                    
            3: alteration_rules[i],
            4: alteration_rules[i],
            5: alteration_rules[i],
            6: alteration_rules[i],
            7: alteration_rules[i],
            8: alteration_rules[i],
            9: alteration_rules[i]
        })

Fetch data from source. Meta should match configuration above

In [3]:
df_source = pd.read_csv('prediction_input.csv', header=0, names=DATASOURCE_COLUMNS)
df_source.FindCode = df_source.FindCode.astype(str).str.lower()
df_source.Name = df_source.Name.astype(str).str.lower()
df_source.Phone = df_source.Phone.astype(str).str.lower()
df_source.Fax = df_source.Fax.astype(str).str.lower()
df_source.OtherPhone = df_source.OtherPhone.astype(str).str.lower()
df_source.Email = df_source.Email.astype(str).str.lower()
df_source.WebsiteURL = df_source.WebsiteURL.astype(str).str.lower()
df_source.MailingAddressFreeform = df_source.MailingAddressFreeform.astype(str).str.lower()
df_source.MailingAddressCity = df_source.MailingAddressCity.astype(str).str.lower()
df_source.MailingAddressPostalCode = df_source.MailingAddressPostalCode.astype(str).str.lower()
df_source.MailingAddressState = df_source.MailingAddressState.astype(str).str.lower()
df_source.replace(to_replace='none', value='', inplace=True)

df_source = df_source[0:10]

Create predictiondataset

In [4]:
builder = DatasetBuilder(DATASOURCE_COLUMNS, DATASOURCE_INDEX, 
                         ONE_HOT_ENCONDING_COLUMNS, TEXT_METRICS, 
                         PASS_THROUGH_COMULNS, COLUMN_ALTERATIION_RULES,
                         HIGH_IMPORTANCE_COLUMNS, 8, logging.DEBUG)
predicting_df = builder.generatePredictionDataset(df_source)

2020-04-17 21:40:04,561 - root - INFO - Column: WebsiteURL_x, ratio metric took: 1.1682307720184326 seconds
2020-04-17 21:40:05,707 - root - INFO - Column: WebsiteURL_x, partial_ratio metric took: 1.144033670425415 seconds
2020-04-17 21:40:06,898 - root - INFO - Column: WebsiteURL_x, token_sort_ratio metric took: 1.1912016868591309 seconds
2020-04-17 21:40:08,023 - root - INFO - Column: WebsiteURL_x, token_set_ratio metric took: 1.124622106552124 seconds
2020-04-17 21:40:09,146 - root - INFO - Column: WebsiteURL_x, distance metric took: 1.1221187114715576 seconds
2020-04-17 21:40:10,267 - root - INFO - Column: WebsiteURL_x, l_ratio metric took: 1.1205530166625977 seconds
2020-04-17 21:40:11,516 - root - INFO - Column: WebsiteURL_x, jaro metric took: 1.247572422027588 seconds
2020-04-17 21:40:12,754 - root - INFO - Column: WebsiteURL_x, jaro_winkler metric took: 1.23738431930542 seconds
2020-04-17 21:40:13,896 - root - INFO - Column: WebsiteURL_x, setratio metric took: 1.141677379608154

2020-04-17 21:41:30,487 - root - INFO - Column: Email_x, jaro metric took: 1.266559362411499 seconds
2020-04-17 21:41:31,746 - root - INFO - Column: Email_x, jaro_winkler metric took: 1.2579612731933594 seconds
2020-04-17 21:41:33,097 - root - INFO - Column: Email_x, setratio metric took: 1.3500397205352783 seconds
2020-04-17 21:41:34,314 - root - INFO - Column: Email_x, seqratio metric took: 1.2165799140930176 seconds
2020-04-17 21:41:35,510 - root - INFO - Column: Email_x, longestnumericseq metric took: 1.195997714996338 seconds
2020-04-17 21:41:36,720 - root - INFO - Column: MailingAddressCity_x, ratio metric took: 1.2025458812713623 seconds
2020-04-17 21:41:37,925 - root - INFO - Column: MailingAddressCity_x, partial_ratio metric took: 1.2053279876708984 seconds
2020-04-17 21:41:39,170 - root - INFO - Column: MailingAddressCity_x, token_sort_ratio metric took: 1.2438740730285645 seconds
2020-04-17 21:41:40,344 - root - INFO - Column: MailingAddressCity_x, token_set_ratio metric too

In [5]:
model = RestoreModel("TrainedModel.sav")
predictor = Predictor(model)
predictor.execute(predicting_df, builder.getCompleteDataset())

In [6]:
matches = builder.getCompleteDataset()[
    [e + '_x' for e in DATASOURCE_COLUMNS] + [e + '_y' for e in DATASOURCE_COLUMNS]].iloc[builder.getCompleteDataset().index]

matches[(matches[DATASOURCE_INDEX + '_x'] != matches[DATASOURCE_INDEX + '_y'])]

,FindCode_x,Name_x,Phone_x,Fax_x,OtherPhone_x,Email_x,WebsiteURL_x,MailingAddressFreeform_x,MailingAddressCity_x,MailingAddressPostalCode_x,...,Name_y,Phone_y,Fax_y,OtherPhone_y,Email_y,WebsiteURL_y,MailingAddressFreeform_y,MailingAddressCity_y,MailingAddressPostalCode_y,MailingAddressState_y
1,aaaqs,tony groen,7054354000,,,,,5523 3rd line,alliston,l9r1v2,...,bill priddle,5199236943,,,,,rr 1,badjeros,n0c1a0,ontario
2,aaaqs,tony groen,7054354000,,,,,5523 3rd line,alliston,l9r1v2,...,august luymes,4037822572,,,,,rr 5 stn main,lacombe,t4l2n5,alberta
3,aaaqs,tony groen,7054354000,,,,,5523 3rd line,alliston,l9r1v2,...,stauffer land & livestock,4037463115,4035883015,4035883019,,,rr 3,eckville,t0m0x0,alberta
4,aaaqs,tony groen,7054354000,,,,,5523 3rd line,alliston,l9r1v2,...,morris wood,3063294692,,,mewood@sasktel.net,,gd,grandora,s0k1v0,saskatchewan
5,aaaqs,tony groen,7054354000,,,,,5523 3rd line,alliston,l9r1v2,...,harvex agromart inc,,,,,,2109 b county rd 20,oxford station,k0g1j0,ontario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,aadro,steve rounds,5193492211,5193492248,,,,rr 3,lakeside,n0m2g0,...,morris wood,3063294692,,,mewood@sasktel.net,,gd,grandora,s0k1v0,saskatchewan
95,aadro,steve rounds,5193492211,5193492248,,,,rr 3,lakeside,n0m2g0,...,harvex agromart inc,,,,,,2109 b county rd 20,oxford station,k0g1j0,ontario
96,aadro,steve rounds,5193492211,5193492248,,,,rr 3,lakeside,n0m2g0,...,sturgeon valley fertilizers ltd,7809613088,7809613084,,tom@svfltd.ca,,box 278,legal,t0g1l0,alberta
97,aadro,steve rounds,5193492211,5193492248,,,,rr 3,lakeside,n0m2g0,...,33r-cam farms ltd.,3063692667,,,,,po box 484,bruno,s0k 0s0,saskatchewan
